# Multi Table Synthetic Data

## Data analysis

For multitable we'll be using the strokes table used on the single table analysis and a people table. For the people table each entry has an ID which matches the person_id on the strokes table. This table also has some PII (Personal Identifiable Information) information which is important for us to masquerade.

## Load Data
First, we go to the content folder and get all csv files there.

In [2]:
from sdv.datasets.local import load_csvs

try:
    datasets = load_csvs(folder_name='content/')
except ValueError as e:
    print(e)

Then, we access both table and display the first 20 rows of the people table.

In [3]:
print(datasets.keys())

strokes_table = datasets['strokes']
people_table = datasets['people']
people_table.head(20)

dict_keys(['people', 'strokes'])


,id,name,address,city
0,56420,Marcelo Holmes,49 Walt Whitman Lane,New York
1,51856,Aleena Hahn,"Apple Valley, CA 92307",Los Angeles
2,41097,Jocelyn Hancock,10 West Church St.,Chicago
3,545,Marcel Underwood,"Hastings, MN 55033",Miami
4,37759,Jazlyn Davila,7444 South Pine Dr.,Dallas
5,66333,Teagan Randall,"Malden, MA 02148",Houston
6,70670,Antony Graham,666 Windfall Dr.,Philadelphia
7,20292,Evelyn Becker,"Niagara Falls, NY 14304",Atlanta
8,72784,Arturo Dillon,334 Grove Street,Washington
9,65895,Alyssa Peters,"Moncks Corner, SC 29461",Boston


## Create metadata
We then need to create the metadata object to be used when creating the synthesizer. SDV will detect some information from the table content but it may not be correct. **It's always best to check the metadata and fix whatever needs to be fixed.**

In [4]:
from sdv.metadata import MultiTableMetadata

metadata = MultiTableMetadata()

metadata.detect_table_from_dataframe(
    table_name='strokes',
    data=strokes_table
)

metadata.detect_table_from_dataframe(
    table_name='people',
    data=people_table
)

print('Auto detected data:\n')
metadata

Auto detected data:



{
    "tables": {
        "strokes": {
            "columns": {
                "id": {
                    "sdtype": "numerical"
                },
                "person_id": {
                    "sdtype": "numerical"
                },
                "gender": {
                    "sdtype": "categorical"
                },
                "age": {
                    "sdtype": "numerical"
                },
                "hypertension": {
                    "sdtype": "numerical"
                },
                "heart_disease": {
                    "sdtype": "numerical"
                },
                "ever_married": {
                    "sdtype": "categorical"
                },
                "work_type": {
                    "sdtype": "categorical"
                },
                "Residence_type": {
                    "sdtype": "categorical"
                },
                "avg_glucose_level": {
                    "sdtype": "numerical"
                },
 

## Edit Metadata

### Strokes Table
Bellow, we make a few changes such as:
- Change column id to type id;
- Change column age to a numerical type of integers;
- Change column bmi to a numerical type of floats;
- Change column stroke to a categorical column since the information is either 0 or 1;
- Change hypertension and heart_disease columns to categorical as well for the same reason.

In [5]:
metadata.update_column(
    table_name='strokes',
    column_name='id',
    sdtype='id'
)

metadata.update_column(
    table_name='strokes',
    column_name='person_id',
    sdtype='id'
)

metadata.update_column(
    table_name='strokes',
    column_name='age',
    sdtype='numerical',
    computer_representation="Int64"
)

metadata.update_column(
    table_name='strokes',
    column_name='bmi',
    sdtype='numerical',
    computer_representation="Float"
)

metadata.update_column(
    table_name='strokes',
    column_name='stroke',
    sdtype='categorical',
)

metadata.update_column(
    table_name='strokes',
    column_name='hypertension',
    sdtype='categorical',
)

metadata.update_column(
    table_name='strokes',
    column_name='heart_disease',
    sdtype='categorical',
)

print(metadata)

{
    "tables": {
        "strokes": {
            "columns": {
                "id": {
                    "sdtype": "id"
                },
                "person_id": {
                    "sdtype": "id"
                },
                "gender": {
                    "sdtype": "categorical"
                },
                "age": {
                    "sdtype": "numerical",
                    "computer_representation": "Int64"
                },
                "hypertension": {
                    "sdtype": "categorical"
                },
                "heart_disease": {
                    "sdtype": "categorical"
                },
                "ever_married": {
                    "sdtype": "categorical"
                },
                "work_type": {
                    "sdtype": "categorical"
                },
                "Residence_type": {
                    "sdtype": "categorical"
                },
                "avg_glucose_level": {
                

### People Table
Bellow, we make a few changes to people type such as:
- Change column id to type id;
- Change column name to type name and mark as PII;
- Change column address to type address and mark as PII;
- Change column city to type city and mark as PII.

We need to add pii to the columns because they are personable identifiable information.

In [6]:
metadata.update_column(
    table_name='people',
    column_name='id',
    sdtype='id'
)

metadata.update_column(
    table_name='people',
    column_name='name',
    sdtype='name',
    pii=True
)

metadata.update_column(
    table_name='people',
    column_name='address',
    sdtype='address',
    pii=True
)

metadata.update_column(
    table_name='people',
    column_name='city',
    sdtype='city',
    pii=True
)

print(metadata)

{
    "tables": {
        "strokes": {
            "columns": {
                "id": {
                    "sdtype": "id"
                },
                "person_id": {
                    "sdtype": "id"
                },
                "gender": {
                    "sdtype": "categorical"
                },
                "age": {
                    "sdtype": "numerical",
                    "computer_representation": "Int64"
                },
                "hypertension": {
                    "sdtype": "categorical"
                },
                "heart_disease": {
                    "sdtype": "categorical"
                },
                "ever_married": {
                    "sdtype": "categorical"
                },
                "work_type": {
                    "sdtype": "categorical"
                },
                "Residence_type": {
                    "sdtype": "categorical"
                },
                "avg_glucose_level": {
                

Then we need to connect the two tables

In [7]:
metadata.set_primary_key(
    table_name='strokes',
    column_name='id'
)

metadata.set_primary_key(
    table_name='people',
    column_name='id'
)

metadata.add_relationship(
    parent_table_name='people',
    child_table_name='strokes',
    parent_primary_key='id',
    child_foreign_key='person_id'
)

print(metadata)

{
    "tables": {
        "strokes": {
            "primary_key": "id",
            "columns": {
                "id": {
                    "sdtype": "id"
                },
                "person_id": {
                    "sdtype": "id"
                },
                "gender": {
                    "sdtype": "categorical"
                },
                "age": {
                    "sdtype": "numerical",
                    "computer_representation": "Int64"
                },
                "hypertension": {
                    "sdtype": "categorical"
                },
                "heart_disease": {
                    "sdtype": "categorical"
                },
                "ever_married": {
                    "sdtype": "categorical"
                },
                "work_type": {
                    "sdtype": "categorical"
                },
                "Residence_type": {
                    "sdtype": "categorical"
                },
                "avg_g

## Create Synthesizer
Having created the metadata object we then needed to create the synthesizer which will be trained to generate the synthetic data. Here we use the only possible (excluding the enterprise edition) synthesizer - HMA. Note that you can configure which synthesizer each table uses.

In [8]:
from sdv.multi_table import HMASynthesizer

synthesizer = HMASynthesizer(metadata)
synthesizer.fit(datasets)

We cannot define number of rows when using multitable but we can define a scale:
- **<1** : Shrink the data by the specified percentage. For example, 0.9 will create synthetic data that is roughly 90% of the size of the original data.;
- **=1** : Don't scale the data. The model will create synthetic data that is roughly the same size as the original data.;
- **\>1** : Scale the data by the specified factor. For example, 2.5 will create synthetic data that is roughly  2.5x the size of the original data.;

In [9]:
synthetic_data = synthesizer.sample(
    scale=2.5
)

synthetic_data

{'people':           id                name   
 0          0       Brent Collins  \
 1          1         Carlos Mata   
 2          2  Mrs. Crystal Blair   
 3          3    Nathaniel Murphy   
 4          4    Shannon Mitchell   
 ...      ...                 ...   
 11995  11995     Misty Dominguez   
 11996  11996       Roberto Brown   
 11997  11997       Deborah Smith   
 11998  11998       William Mason   
 11999  11999          Todd Price   
 
                                                  address                city  
 0      6157 Clark Rest\nSouth Christophershire, MT 67360        New Johnfurt  
 1                       PSC 8737, Box 1740\nAPO AA 35924           Jacobland  
 2              922 Smith Union\nPort Shawnbury, DE 51299           Holdenton  
 3             3937 William Mount\nCohenborough, KY 49299         West Mariah  
 4              09073 Manning Vista\nGravesport, MT 89179   South Thomashaven  
 ...                                                  ...       

If you search for this info on the original data you can see that it's not there since we marked name, address and city as PII.

<h2>Evaluation</h2>

In [9]:
from sdv.evaluation.multi_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=datasets,
    synthetic_data=synthetic_data,
    metadata=metadata)

Creating report: 100%|██████████| 5/5 [00:00<00:00, 15.89it/s]



Overall Quality Score: 91.12%

Properties:
Column Shapes: 90.35%
Column Pair Trends: 83.01%
Parent Child Relationships: 100.0%


In [10]:
from sdv.evaluation.multi_table import run_diagnostic

diagnostic_report = run_diagnostic(
    real_data=datasets,
    synthetic_data=synthetic_data,
    metadata=metadata)

Creating report: 100%|██████████| 4/4 [00:28<00:00,  7.15s/it]


DiagnosticResults:

SUCCESS:
✓ The synthetic data covers over 90% of the numerical ranges present in the real data
✓ The synthetic data covers over 90% of the categories present in the real data
✓ Over 90% of the synthetic rows are not copies of the real data
✓ The synthetic data follows over 90% of the min/max boundaries set by the real data


In [11]:
diagnostic_report.get_results()

{'SUCCESS': ['The synthetic data covers over 90% of the numerical ranges present in the real data',
  'The synthetic data covers over 90% of the categories present in the real data',
  'Over 90% of the synthetic rows are not copies of the real data',
  'The synthetic data follows over 90% of the min/max boundaries set by the real data'],
 'WARNING': [],
 'DANGER': []}

In [12]:
diagnostic_report.get_properties()

{'Coverage': 0.9733928657693466, 'Synthesis': 1.0, 'Boundaries': 1.0}

## PII Locales
It's also possible to generate data from certain languages on certain countries. For example if we want only french canadian names here's how to proceed.

We create a AnonymizedFaker which receives the provider and function names from the Python Faker Library and a locales array with possible **language_countries**. And then we retrain the synthesizer for our changes to be learned.

In [13]:
from rdt.transformers.pii import AnonymizedFaker

synthesizer.update_transformers(table_name="people", column_name_to_transformer={
    'name': AnonymizedFaker(provider_name='person', function_name='name', locales=['fr_CA'])
})

synthesizer.fit(datasets)

/Users/vascopais/Library/Caches/pypoetry/virtualenvs/synthetic-data-EqHpLbmO-py3.10/lib/python3.10/site-packages/sdv/single_table/base.py:292: UserWarning: For this change to take effect, please refit the synthesizer using `fit`.
  warnings.warn(msg, UserWarning)
/Users/vascopais/Library/Caches/pypoetry/virtualenvs/synthetic-data-EqHpLbmO-py3.10/lib/python3.10/site-packages/sdv/single_table/base.py:292: UserWarning: For this change to take effect, please refit the synthesizer using `fit`.
  warnings.warn(msg, UserWarning)
/Users/vascopais/Library/Caches/pypoetry/virtualenvs/synthetic-data-EqHpLbmO-py3.10/lib/python3.10/site-packages/sdv/single_table/base.py:292: UserWarning: For this change to take effect, please refit the synthesizer using `fit`.
  warnings.warn(msg, UserWarning)


Then we generate the data once again (this time a smaller sample). And we can see that the names on the people table are now french canadian names.

In [14]:
synthesizer.sample(scale=0.01)

{'people':     id                         name   
 0    0           Emmanuelle Poirier  \
 1    1                  Maude Blais   
 2    2     Caroline Trottier-Lemire   
 3    3                Édouard Soucy   
 4    4       Alexis-Emmanuel Séguin   
 5    5             Pénélope Gervais   
 6    6   Dorothée Marcoux-Larivière   
 7    7                Julien Larose   
 8    8              Michèle Couture   
 9    9              Susanne St-Onge   
 10  10        Jeannine-Manon Daigle   
 11  11       Martin Gingras-Provost   
 12  12     Emmanuel Dionne-Rodrigue   
 13  13               Michèle Bérubé   
 14  14               Josette Bérubé   
 15  15      Juliette-Sylvie Bernier   
 16  16               Pauline Nadeau   
 17  17            Marcel Létourneau   
 18  18              Jacques Fortier   
 19  19                 Louis Lepage   
 20  20             Céline Arsenault   
 21  21           Xavier-Jean Larose   
 22  22       William Bédard-Germain   
 23  23                 Maude 

## Conclusion

This time on the multi table generation we got a better coverage percentage on the quality report. This may be because of:
- Having 2.5x the synthetic data compared to the single table example;
- Using the GaussianCopula model for each table with HMA on multi table data generation vs. using the GaussianCopula model with FastML preset on single table data generation. The latter uses a normal distribution, no enforced rounding and the categorical transformer is a FrequencyEncoder instead of a LabelEncoder. More about these encoders on the [source code](https://github.com/sdv-dev/RDT/blob/master/rdt/transformers/categorical.py).

It's also important to note that even though the SDV library has a tool to evaluate the quality of the data created we should have a third party library to also test this quality in order to be as unbiased as possible.

Now we know how to create synthetic data and how quick and secure it can be.